In [1]:
# Script: KRG_EstMI_NonLinear.ipynb
# Author: Robert Kissell, Ph.D.
# Company: Kissell Research Group

In [2]:
# Step 1. Import all Packages: Pandas, Numpy, and Statistics
import pandas as pd
import numpy as np
#import statistics as st
import scipy
#from scipy import stats
import scipy.optimize as spo
from scipy.optimize import minimize
from scipy.optimize import fsolve

In [3]:
# Step 2. User Inputs
InputPath="c:/Algo/"
OutputPath="c:/Algo/"

MIDataFile="MIData_KRG.csv"
MIParameterFile="MiParameter.csv"


In [4]:
# Step 3. Read MIData file
MI_df = pd.read_csv(InputPath+MIDataFile)
MI_df.head()

,Date,Size,Volatility,POV,Time,Cost
0,8/14/2020,0.0927,0.498829,0.0303,2.966706,-0.357895
1,8/14/2020,0.0568,0.344376,0.0333,1.648906,171.617139
2,8/14/2020,0.0534,0.193020,0.0415,1.233347,-26.072693
3,8/14/2020,0.0580,0.471967,0.0478,1.155389,260.844290
4,8/14/2020,0.1500,0.337130,0.0538,2.638104,126.922787


In [5]:
# Step 4. Define Variables from MIData file
Size = MI_df['Size'].to_numpy()
Volatility = MI_df['Volatility'].to_numpy()
POV = MI_df['POV'].to_numpy()
Cost = MI_df['Cost'].to_numpy()


In [6]:
# Step 5. Define function to calculate Non-Linear Objective Function
def optimize1(initial_guess, Size, POV, Volatility, Cost):
    a1, a2, a3, a4, b1 = initial_guess
    return np.sum(np.square((a1 * (Size ** a2) * (Volatility ** a3) * (b1 * (POV ** a4) + (1 - b1))) - Cost))


In [8]:
# Step 6. Initial Guess for Non-Linear Optimization
a1 = 1000
a2 = 0.5
a3 = 0.5
a4 = 0.5
b1 = 0.95
initial_guess = [a1, a2, a3, a4, b1]

# Set Upper and Lower Bounds on Constraints
LB = [0.0001, 0.0001, 0.0001, 0.0001, 0.00001] 
UB = [2000, 2.0, 2.0, 2.0, 1]

MyBounds = []
i = 0 
for i in range(0,len(LB)):
    MyBounds.append((LB[i], UB[i]))
    i = i+1

result = minimize(optimize1, initial_guess, bounds = MyBounds, args = (Size, POV, Volatility, Cost))
result

      fun: 4945541183.819699
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([    0.        , 54836.27322222, 46157.83668213, 53405.7614504 ,
       93650.81740052])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 174
      nit: 25
     njev: 29
   status: 0
  success: True
        x: array([8.82990307e+02, 3.53993552e-01, 7.55580954e-01, 8.25067453e-01,
       9.63922619e-01])

In [9]:
# Step 7. Save MI Parameters to output file
a1 = result.x.tolist()[0]
a2 = result.x.tolist()[1]
a3 = result.x.tolist()[2]
a4 = result.x.tolist()[3]
b1 = result.x.tolist()[4]

new_a1 = pd.Series(result.x.tolist()[0])
new_a2 = pd.Series(result.x.tolist()[1])
new_a3 = pd.Series(result.x.tolist()[2])
new_a4 = pd.Series(result.x.tolist()[3])
new_b1 = pd.Series(result.x.tolist()[4])
    #Optimized_SSE = pd.Series(np.sum(np.square((a1 * (Size ** a2) * (Volatility ** a3) * (b1 * (POV ** a4) + (1 - b1))) - Cost)))
    #MiConstraints = pd.concat([new_a1 ,new_a2, new_a3, new_a4, new_b1, Optimized_SSE], axis =1)
    #MiConstraints.columns = [['a1', 'a2', 'a3', 'a4', 'b1', 'Optimized_SSE']]
MiParameters = pd.concat([new_a1 ,new_a2, new_a3, new_a4, new_b1], axis =1)
MiParameters.columns = [['a1', 'a2', 'a3', 'a4', 'b1']]
MiParameters = MiParameters.transpose()
MiParameters.columns = ['Value']
MiParameters.index.rename('Parameter', inplace=True)

#MiParameters.to_csv(OutputPath+'MiParameters.csv', index = True, header = True)
MiParameters.to_csv(OutputPath+MIParameterFile, index = True, header = True)

In [10]:
# Step 8. Display MI Parameters to screen
MiParameters

,Value
Parameter,
a1,882.990307
a2,0.353994
a3,0.755581
a4,0.825067
b1,0.963923
